# Libraries and modules


In [1]:
import pandas as pd 
from datetime import datetime
from FR.internal import selectdfQ, regexInternalData as rd, scrapedInternalPage
from FR.tools import savedata as sd, deltempfiles as dtf
from FR.directories import master_directory, directory_bat_file 

# __ENVIRONMENT__

In [2]:
# Variables 
month = 'FEB'       # Update each month
year = '2024'       # Update each month

# Constants  
OPERATION = 'Rental'
WEB = 'FR'

# Master directory
master_dir = master_directory()
master_dir = f'{master_dir}/{WEB}'

# IMPORT
# Directories to import files 
moni_dir_imp = f'{master_dir}/FRONTPAGE/MONITORING/{year}/{month}'

# import files 
###############################################################################################################
fl_master_imp = f'{WEB}_{month}{year}_{OPERATION}_Front.csv'

inputfile = f'{moni_dir_imp}/{fl_master_imp}' 

# EXPORT
# Directories to export files 
result_dir = f'{master_dir}/INTERNAL/JOIN'

# Export files 
###############################################################################################################
# This files is going to be joined for create the master file, which contains all the data for the fron page
fl_pre = f'{WEB}_join_{OPERATION}INT_{month}{year}_'


# Ruta al archivo .bat
ruta_bat = directory_bat_file()
ruta_bat = f'{ruta_bat}/deltemporal_exect.bat'

# Read frontpage df 

In [3]:
df = pd.read_csv(f'{inputfile}', sep = ';', encoding = 'utf-16')

## Choosing the df 


In [4]:
dfQ3, _ = selectdfQ(df, 3, 'Arriendo')

Usted seleccionó dfQ3 con: 9554 obs


# Split data 

In [5]:
# Split principal df - Q1
# This is for close the driver each 500 urls or internal pages processes
#############################################################################################################
num_dfs = len(dfQ3) // 500 + (1 if len(dfQ3) % 500 != 0 else 0)
dfs = {f'{fl_pre}_Q3{i}': dfQ3.iloc[i:i+500] for i in range(0, len(dfQ3), 500)}
print(f'You created {num_dfs} DataFrames from the principal df')

Se crearon 20 DataFrames para df_inmobiliaria:


# __Scraping__

In [ ]:
# Initialize the list for save the data
inmueble_slow_resultados = []    

try:
    start_time = datetime.now()
    print(f"The process started at {start_time.strftime('%H:%M')} of {start_time.strftime('%d/%m/%Y')}")

    # Itearte for each df and scraped the data of each internal page
    for df_key, df_value in dfs.items():
        print(f"Realizando scraping en {df_key}...")
        scraped_data = scrapedInternalPage(df_value)
        inmueble_slow_resultados.extend(scraped_data) 
        
        # Use the function regex for have the variables 
        processed_data = rd(scraped_data)
        sd(processed_data, f'{result_dir}/{df_key}')
        
        # Delete the temporal files 
        dtf(ruta_bat)

    end_time = datetime.now()
    print(f"The process ended at {end_time.strftime('%H:%M')} of {end_time.strftime('%d/%m/%Y')}")
    
except Exception as e:
    print(f"Error: {e}")
